<a href="https://colab.research.google.com/github/TNSA-SOFTWARES/GPFLLaT-1/blob/main/TNSA_SREENIVAS_MODEL_GPFLLaT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets tensorflow

In [5]:
import tensorflow as tf
from keras.layers import Layer, LSTM, Dense, Attention, Embedding, Concatenate
import numpy as np
from datasets import load_dataset


In [ ]:
datasets = {
    "math_word_problems": load_dataset("microsoft/orca-math-word-problems-200k"),
    "classification_test": load_dataset("DIBT/10k_prompts_ranked"),
    "text_translation": load_dataset("Open-Orca/OpenOrca"),
    "text_generation": load_dataset("DIBT/10k_prompts_ranked"),
    "text2text_generation": load_dataset("openai_humaneval"),
    "sentence_similarity": load_dataset("ymoslem/Law-StackExchange"),
    "data_augmentation": load_dataset("databricks/databricks-dolly-15k")
}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/10331 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
class PositionalEncoding(Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / np.power(10000., (2 * (i // 2)) / np.float32(d_model))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            np.arange(position)[:, np.newaxis],
            np.arange(d_model)[np.newaxis, :],
            d_model
        )

        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]

        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


In [8]:
class CustomModel(tf.keras.Model):
    def __init__(self, d_model, num_layers, num_heads, dff, max_seq_len, vocab_size=10000, dropout_rate=0.1):
        super(CustomModel, self).__init__()
        self.embedding = Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(max_seq_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.lstm_layers = [LSTM(d_model, return_sequences=True) for _ in range(num_layers)]
        self.attention = Attention()
        self.concatenation = Concatenate()
        self.dense = Dense(vocab_size)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.pos_encoding(x)
        x = self.dropout(x)

        for layer in self.lstm_layers:
            x = layer(x)

        context_vector = self.attention([x, x])
        x = self.concatenation([x, context_vector])
        x = self.dense(x)

        return x


In [28]:
# Instantiate the custom model
custom_model = CustomModel(
    d_model=256*1000,
    num_layers=4*1000,
    num_heads=8*1000,
    dff=512*1000,
    max_seq_len=100*1000
)

# Define dummy training data (replace with your actual training data)
dummy_inputs = np.random.randint(0, 10000, (8, 100))  # Example dummy inputs with vocab size of 10,000


In [ ]:
# Print model summary
print("Srenivas Model Summary:")
custom_model.build(input_shape=(None, None))
custom_model.summary()

# Define dummy training data (replace with your actual training data)
dummy_inputs = np.random.randint(0, 10000, (8, 100))  # Example dummy inputs with vocab size of 10,000

# Calculate disk space required for model parameters
disk_space_calculator = DiskSpace()
disk_space_calculator.model = custom_model
disk_space_calculator.on_train_end()

Srenivas Model Summary:


In [ ]:
# Compile the model
custom_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model with dummy inputs
custom_model.fit(dummy_inputs, dummy_inputs, batch_size=8, epochs=3)


In [12]:
# Save the trained model as TensorFlow SavedModel
tf.saved_model.save(custom_model, 'SREENIVAS-MODEL-GPFLLaT-1-N-GEN-2')



In [1]:
%ls

sample_data/
